### 1. Installa e importa i pacchetti necessari

In [ ]:
!pip install transformers

In [ ]:
import  os
from    transformers       import pipeline
from    nltk               import ngrams
import  warnings
warnings.filterwarnings( 'ignore' )

In [ ]:
from IPython.display  import HTML, display
def set_css():
    display( HTML( '''<style> pre { white-space: pre-wrap; } </style>''' ) )
get_ipython().events.register( 'pre_run_cell', set_css )

### 2. Scarica il modello da Hugging Face

In [ ]:
def get_model( model_name ):
    return pipeline( "text-generation", model=model_name, device=-1 )

model_name  = "distilgpt2"
model       = get_model( model_name )

### 3. Imposta la completion del modello

In [ ]:
def completion( model, prompt, temp ):
    res     = model(
        prompt,
        min_length              = 100,
        max_new_tokens          = 400,
        num_return_sequences    = 3,
        temperature             = temp
    )
    return [ t[ 'generated_text' ] for t in res ]

In [ ]:
def check_res( res, max_reps=None, min_len=100 ):
    if len( res ) < min_len:
        # completion too short
        return False
        
    words       = res.split()
    nw          = len( words )
    if max_reps is None:
        # a reasonable maximum number of repetitions
        max_reps    = nw // 25

    reps        = []
    # check for 3-grams, 4-grams, and 5-grams
    for n in range( 3, 6 ):
        ng      = list( ngrams( words, n=n ) )
        reps    += [ ng.count( w ) for w in ng ]
        
    if max( reps ) > max_reps:
        # too many repetitions
        return False
    return True

In [ ]:
def print_res( res, max_rep ):
    for i, r in enumerate( res ):
        print( '=' * 40 )
        print( "RESULT", i+1 )
        print( "Too many repetitions?", "No" if check_res( r, max_reps ) else "Yes" )
        print( '=' * 40 )
        print( '\n' + r + '\n\n' )

### 4. Testa il modello con diverse temperature

In [ ]:
prompt    = "Once upon a time, in a land far far away, there was"
temp      = 0.8
max_reps  = 10

res       = completion( model, prompt, temp )
print_res( res, max_reps )

In [ ]:
prompt    = "They were locked together in the room. She knew one of them was the murderer. Therefore, she"
temp      = 1.2
max_reps  = 10

res       = completion( model, prompt, temp )
print_res( res, max_reps )